In [48]:
import os
import re
import numpy as np
import pandas as pd
import unicodedata
import math
import copy
from zipfile import ZipFile
import random

In [69]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import BaggingClassifier

from sklearn.ensemble import RandomForestClassifier

from sklearn.ensemble import GradientBoostingClassifier

In [50]:
import warnings
warnings.filterwarnings("ignore")

In [51]:
os.mkdir("/content/rhugaved_data")
os.chdir("/content/rhugaved_data")

After running above part, upload the zip folder in /content/rhugaved_data then run next cells

In [52]:
for filename in os.listdir("/content/rhugaved_data"):
    if not os.path.isdir(filename):
        with ZipFile(filename, 'r') as zip:
            # extracting all the files
            zip.extractall()

In [53]:
os.chdir("/content/rhugaved_data/all_data")

In [54]:
!ls

test_c1000_d1000.csv  train_c1000_d1000.csv  valid_c1000_d1000.csv
test_c1000_d100.csv   train_c1000_d100.csv   valid_c1000_d100.csv
test_c1000_d5000.csv  train_c1000_d5000.csv  valid_c1000_d5000.csv
test_c1500_d1000.csv  train_c1500_d1000.csv  valid_c1500_d1000.csv
test_c1500_d100.csv   train_c1500_d100.csv   valid_c1500_d100.csv
test_c1500_d5000.csv  train_c1500_d5000.csv  valid_c1500_d5000.csv
test_c1800_d1000.csv  train_c1800_d1000.csv  valid_c1800_d1000.csv
test_c1800_d100.csv   train_c1800_d100.csv   valid_c1800_d100.csv
test_c1800_d5000.csv  train_c1800_d5000.csv  valid_c1800_d5000.csv
test_c300_d1000.csv   train_c300_d1000.csv   valid_c300_d1000.csv
test_c300_d100.csv    train_c300_d100.csv    valid_c300_d100.csv
test_c300_d5000.csv   train_c300_d5000.csv   valid_c300_d5000.csv
test_c500_d1000.csv   train_c500_d1000.csv   valid_c500_d1000.csv
test_c500_d100.csv    train_c500_d100.csv    valid_c500_d100.csv
test_c500_d5000.csv   train_c500_d5000.csv   valid_c500_d5000.csv


In [82]:
dt = [['c1000_d5000', 'gini', 'random', 7, 'auto'], ['c1000_d1000', 'entropy', 'best', 7, 'auto'], ['c1000_d100', 'gini', 'random', 5, 'auto'], ['c1500_d5000', 'gini', 'best', 15, 'auto'], ['c1500_d1000', 'entropy', 'best', 7, 'auto'], ['c1500_d100', 'gini', 'best', 5, 'auto'], ['c1800_d5000', 'entropy', 'random', 15, 'auto'], ['c1800_d1000', 'entropy', 'random', 15, 'log2'], ['c1800_d100', 'gini', 'random', 7, 'log2'], ['c300_d5000', 'gini', 'best', 7, 'auto'], ['c300_d1000', 'gini', 'best', 5, 'auto'], ['c300_d100', 'gini', 'random', 2, 'log2'], ['c500_d5000', 'entropy', 'best', 7, 'auto'], ['c500_d1000', 'entropy', 'best', 7, 'auto'], ['c500_d100', 'gini', 'best', 5, 'log2']]


bagg = [['c1000_d5000', 'gini', 'best', 15, 'log2', 500], ['c1000_d1000', 'gini', 'best', 15, 'log2', 500], ['c1000_d100', 'gini', 'best', 10, 'log2', 500], ['c1500_d5000', 'gini', 'best', 2, 'log2', 500], ['c1500_d1000', 'gini', 'best', 2, 'log2', 500], ['c1500_d100', 'gini', 'best', 2, 'auto', 100], ['c1800_d5000', 'gini', 'best', 2, 'log2', 100], ['c1800_d1000', 'gini', 'best', 2, 'auto', 500], ['c1800_d100', 'gini', 'best', 2, 'auto', 50], ['c300_d5000', 'entropy', 'best', 10, 'log2', 500], ['c300_d1000', 'entropy', 'best', 5, 'log2', 500], ['c300_d100', 'gini', 'random', 7, 'log2', 500], ['c500_d5000', 'entropy', 'best', 10, 'log2', 500], ['c500_d1000', 'entropy', 'random', 7, 'log2', 500], ['c500_d100', 'gini', 'best', 2, 'log2', 500]]

rf = [['c1000_d5000', 'entropy', 500, 20, 'log2'], ['c1000_d1000', 'gini', 500, 15, 'log2'], ['c1000_d100', 'gini', 500, 5, 'log2'], ['c1500_d5000', 'entropy', 100, 10, 'log2'], ['c1500_d1000', 'gini', 50, 5, 'log2'], ['c1500_d100', 'gini', 50, 2, 'auto'], ['c1800_d5000', 'gini', 50, 5, 'log2'], ['c1800_d1000', 'gini', 50, 2, 'auto'], ['c1800_d100', 'gini', 10, 5, 'auto'], ['c300_d5000', 'entropy', 500, 20, 'auto'], ['c300_d1000', 'entropy', 500, 10, 'auto'], ['c300_d100', 'entropy', 500, 10, 'auto'], ['c500_d5000', 'gini', 500, 20, 'log2'], ['c500_d1000', 'gini', 500, 5, 'log2'], ['c500_d100', 'gini', 500, 2, 'log2']]

gb = [['c1000_d5000', 'deviance', 0.1, 500, 'friedman_mse'], ['c1000_d1000', 'exponential', 0.1, 500, 'friedman_mse'], ['c1000_d100', 'deviance', 0.1, 500, 'friedman_mse'], ['c1500_d5000', 'deviance', 0.1, 500, 'friedman_mse'], ['c1500_d1000', 'deviance', 0.1, 500, 'friedman_mse'], ['c1500_d100', 'deviance', 0.1, 500, 'friedman_mse'], ['c1800_d5000', 'deviance', 0.1, 500, 'friedman_mse'], ['c1800_d1000', 'deviance', 0.1, 100, 'squared_error'], ['c1800_d100', 'deviance', 0.1, 500, 'friedman_mse'], ['c300_d5000', 'deviance', 0.1, 500, 'friedman_mse'], ['c300_d1000', 'deviance', 0.1, 500, 'friedman_mse'], ['c300_d100', 'deviance', 0.1, 500, 'friedman_mse'], ['c500_d5000', 'deviance', 0.1, 500, 'friedman_mse'], ['c500_d1000', 'deviance', 0.1, 500, 'friedman_mse'], ['c500_d100', 'exponential', 0.1, 500, 'friedman_mse']]

for f in range(len(dt)):
    print(dt[f][0])
    final_data_management(dt[f][0], dt[f][1:], bagg[f][1:], rf[f][1:], gb[f][1:])

c1000_d5000
DT>>>) Accuracy -> 0.7939 &&& F1 Score -> 0.7943319030036923
BAGG>>>) Accuracy -> 0.9988 &&& F1 Score -> 0.998800719568259
RF>>>) Accuracy -> 0.999 &&& F1 Score -> 0.999
GB>>>) Accuracy -> 1.0 &&& F1 Score -> 1.0
c1000_d1000
DT>>>) Accuracy -> 0.78 &&& F1 Score -> 0.7920604914933836
BAGG>>>) Accuracy -> 0.998 &&& F1 Score -> 0.998001998001998
RF>>>) Accuracy -> 0.998 &&& F1 Score -> 0.998001998001998
GB>>>) Accuracy -> 0.9985 &&& F1 Score -> 0.9985022466300548
c1000_d100
DT>>>) Accuracy -> 0.68 &&& F1 Score -> 0.6701030927835052
BAGG>>>) Accuracy -> 1.0 &&& F1 Score -> 1.0
RF>>>) Accuracy -> 1.0 &&& F1 Score -> 1.0
GB>>>) Accuracy -> 0.98 &&& F1 Score -> 0.98
c1500_d5000
DT>>>) Accuracy -> 0.9185 &&& F1 Score -> 0.9191066997518611
BAGG>>>) Accuracy -> 0.9995 &&& F1 Score -> 0.9994999499949995
RF>>>) Accuracy -> 0.9999 &&& F1 Score -> 0.9998999899989999
GB>>>) Accuracy -> 1.0 &&& F1 Score -> 1.0
c1500_d1000
DT>>>) Accuracy -> 0.884 &&& F1 Score -> 0.8872691933916423
BAGG>>>)

In [79]:
def final_data_management(file_name, dtp, bagp, rfp, gbp):

    col_names = [i for i in range(501)]
    train = test_data = pd.read_csv("train_" + file_name + ".csv", names=col_names)
    tune = test_data = pd.read_csv("valid_" + file_name + ".csv", names=col_names)
    test = test_data = pd.read_csv("test_" + file_name + ".csv", names=col_names)

    train_X = train.iloc[:,:500]
    train_Y = train.iloc[:,500]

    tune_X = tune.iloc[:,:500]
    tune_Y = tune.iloc[:,500]

    test_X = test.iloc[:,:500]
    test_Y = test.iloc[:,500]


    # return train_X, train_Y, tune_X, tune_Y, test_X, test_Y
    final_evaluation(train_X, train_Y, tune_X, tune_Y, test_X, test_Y, dtp, bagp, rfp, gbp)

In [74]:
def final_evaluation(trainX, trainY, tuneX, tuneY, testX, testY, dtp, bagp, rfp, gbp):
    
    # DT
    dtree = DecisionTreeClassifier(random_state=0, criterion=dtp[0], splitter=dtp[1], max_depth=dtp[2], max_features=dtp[3])
    dtree.fit(trainX.append(tuneX), trainY.append(tuneY), sample_weight=None, check_input=True)
    predY = dtree.predict(testX)
    score = accuracy_score(testY, predY)
    f1score = f1_score(testY, predY)
    print(f"DT>>>) Accuracy -> {score} &&& F1 Score -> {f1score}")

    # BAG
    dtree = DecisionTreeClassifier(random_state=0, criterion=bagp[0], splitter=bagp[1], max_depth=bagp[2], max_features=bagp[3])
    dtree.fit(trainX.append(tuneX), trainY.append(tuneY), sample_weight=None, check_input=True)
    bagg = BaggingClassifier(base_estimator=dtree, n_estimators=bagp[4], random_state=0)
    bagg.fit(trainX.append(tuneX), trainY.append(tuneY))
    predY = bagg.predict(testX)
    score = accuracy_score(testY, predY)
    f1score = f1_score(testY, predY)
    print(f"BAGG>>>) Accuracy -> {score} &&& F1 Score -> {f1score}")

    #RF
    randomforest = RandomForestClassifier(random_state=0, criterion=rfp[0], n_estimators=rfp[1], max_depth=rfp[2], max_features=rfp[3])
    randomforest.fit(trainX.append(tuneX), trainY.append(tuneY), sample_weight=None)
    predY = randomforest.predict(testX)
    score = accuracy_score(testY, predY)
    f1score = f1_score(testY, predY)
    print(f"RF>>>) Accuracy -> {score} &&& F1 Score -> {f1score}")

    # GB
    gradientboost = GradientBoostingClassifier(loss=gbp[0], learning_rate=gbp[1], n_estimators=gbp[2], criterion=gbp[3])
    gradientboost.fit(trainX.append(tuneX), trainY.append(tuneY), sample_weight=None)
    predY = gradientboost.predict(testX)
    score = accuracy_score(testY, predY)
    f1score = f1_score(testY, predY)
    print(f"GB>>>) Accuracy -> {score} &&& F1 Score -> {f1score}")

In [55]:
def data_management(file_name):

    col_names = [i for i in range(501)]
    train = test_data = pd.read_csv("train_" + file_name + ".csv", names=col_names)
    tune = test_data = pd.read_csv("valid_" + file_name + ".csv", names=col_names)
    test = test_data = pd.read_csv("test_" + file_name + ".csv", names=col_names)

    train_X = train.iloc[:,:500]
    train_Y = train.iloc[:,500]

    tune_X = tune.iloc[:,:500]
    tune_Y = tune.iloc[:,500]

    test_X = test.iloc[:,:500]
    test_Y = test.iloc[:,500]

    # print(train_X)
    # print(tune_X)
    # print(train_X.append(tune_X))

    # print(train_Y)
    # print(tune_Y)
    # print(train_Y.append(tune_Y))

    print("\n\nDECISION TRESS\n\n")
    DTmodel(train_X, train_Y, tune_X, tune_Y, test_X, test_Y)
    print("\n\nBAGGING\n\n")
    BAGmodel(train_X, train_Y, tune_X, tune_Y, test_X, test_Y)
    print("\n\nRANDOM FORESTS\n\n")
    RFmodel(train_X, train_Y, tune_X, tune_Y, test_X, test_Y)
    print("\n\nGRADIENT BOOSTING\n\n")
    GBmodel(train_X, train_Y, tune_X, tune_Y, test_X, test_Y)

    # return train_X, train_Y, tune_X, tune_Y, test_X, test_Y


In [56]:
def DTmodel(trainX, trainY, tuneX, tuneY, testX, testY):
    criterion = ["gini", "entropy"]
    splitter = ["best", "random"]
    max_depth = [2, 5, 7, 10, 15, 20]
    max_features =  ["auto", "sqrt", "log2"]
    i = 0

    max_score = 0
    for c in criterion:
        for s in splitter:
            for d in max_depth:
                for f in max_features:
                    i += 1
                    dtree = DecisionTreeClassifier(random_state=0, criterion=c, splitter=s, max_depth=d, max_features=f)
                    dtree.fit(trainX, trainY, sample_weight=None, check_input=True)
                    score = dtree.score(tuneX, tuneY, sample_weight=None)
                    if score > max_score:
                        max_score = score
                        max_criterion = c
                        max_splitter = s
                        max_max_depth = d
                        max_max_features = f
                    print(f"{i}) criterion: {c}, splitter: {s}, max_depth: {d}, max_features: {f} -> {score}")
        

    
    print(f">>>) criterion: {max_criterion}, splitter: {max_splitter}, max_max_depth: {max_max_depth}, max_max_features: {max_max_features} -> {max_score}")

    dtree = DecisionTreeClassifier(random_state=0, criterion=c, splitter=s, max_depth=d, max_features=f)
    dtree.fit(trainX.append(tuneX), trainY.append(tuneY), sample_weight=None, check_input=True)
    score = dtree.score(testX, testY, sample_weight=None)
    print(f">>>) Train+Tune: criterion: {max_criterion}, splitter: {max_splitter}, max_max_depth: {max_max_depth}, max_max_features: {max_max_features} -> {score}")




In [57]:
def BAGmodel(trainX, trainY, tuneX, tuneY, testX, testY):
    criterion = ["gini", "entropy"]
    splitter = ["best", "random"]
    max_depth = [2, 5, 7, 10, 15, 20]
    max_features =  ["auto", "sqrt", "log2"]
    n_estimators = [10, 50, 100, 500]
    i = 0

    max_score = 0
    for c in criterion:
        for s in splitter:
            for d in max_depth:
                for f in max_features:
                    i += 1
                    dtree = DecisionTreeClassifier(random_state=0, criterion=c, splitter=s, max_depth=d, max_features=f)
                    dtree.fit(trainX, trainY, sample_weight=None, check_input=True)
                    for n in n_estimators:
                        bagg = BaggingClassifier(base_estimator=dtree, n_estimators=n, random_state=0).fit(trainX, trainY)
                        score = bagg.score(tuneX, tuneY)
                        if score > max_score:
                            max_score = score
                            max_criterion = c
                            max_splitter = s
                            max_max_dept = d
                            max_max_features = f
                            max_estimators = n
                        print(f"{i}) criterion: {c}, splitter: {s}, max_depth: {d}, max_features: {f} -> {score}")
        

    print(f">>>) criterion: {max_criterion}, splitter: {max_splitter}, max_max_dept: {max_max_dept}, max_max_features: {max_max_features}, max_estimators: {max_estimators} -> {max_score}")
    dtree = DecisionTreeClassifier(random_state=0, criterion=c, splitter=s, max_depth=d, max_features=f)
    dtree.fit(trainX.append(tuneX), trainY.append(tuneY), sample_weight=None, check_input=True) 
    bagg = BaggingClassifier(base_estimator=dtree, n_estimators=n, random_state=0).fit(trainX.append(tuneX), trainY.append(tuneY))
    score = bagg.score(testX, testY)
    print(f">>>) Train+Tune: criterion: {max_criterion}, splitter: {max_splitter}, max_max_dept: {max_max_dept}, max_max_features: {max_max_features}, max_estimators: {max_estimators} -> {score}")




In [58]:
def RFmodel(trainX, trainY, tuneX, tuneY, testX, testY):
    criterion = ["gini", "entropy"]
    n_estimators = [10, 50, 100, 500]
    max_depth = [2, 5, 10, 15, 20]
    max_features =  ["auto", "sqrt", "log2"]
    i = 0

    max_score = 0
    for c in criterion:
        for s in n_estimators:
            for d in max_depth:
                for f in max_features:
                    i += 1
                    randomforest = RandomForestClassifier(random_state=0, criterion=c, n_estimators=s, max_depth=d, max_features=f)
                    randomforest.fit(trainX, trainY, sample_weight=None)
                    score = randomforest.score(testX, testY, sample_weight=None)
                    if score > max_score:
                        max_score = score
                        max_criterion = c
                        max_estimators = s
                        max_max_dept = d
                        max_max_features = f
                    print(f"{i}) criterion: {c}, n_estimators: {s}, max_depth: {d}, max_features: {f} -> {score}")
        

    print(f">>>) criterion: {max_criterion}, n_estimators: {max_estimators}, max_max_dept: {max_max_dept}, max_max_features: {max_max_features} -> {max_score}")
    randomforest = RandomForestClassifier(random_state=0, criterion=c, n_estimators=s, max_depth=d, max_features=f)
    randomforest.fit(trainX.append(tuneX), trainY.append(tuneY), sample_weight=None)
    score = randomforest.score(testX, testY, sample_weight=None)
    print(f">>>) Train+Tune: criterion: {max_criterion}, n_estimators: {max_estimators}, max_max_dept: {max_max_dept}, max_max_features: {max_max_features} -> {score}")



In [59]:
def GBmodel(trainX, trainY, tuneX, tuneY, testX, testY):
    # criterion = ["gini", "entropy"]
    # splitter = ["best", "random"]
    # n_estimators = [10, 50, 100, 500, 1000]
    # max_depth = [2, 5, 7, 10, 12, 15, 18, 20]
    # max_features =  ["auto", "sqrt", "log2"]
    

    # GB
    loss = ["deviance", "exponential"]
    lr = [0.001, 0.01, 0.1]
    n_estimators = [10, 50, 100, 500]
    criterion = ["friedman_mse", "squared_error", "mse"]
    i = 0
    # for c in criterion:
    #     for s in n_estimators:
    #         for d in max_depth:
    #             for f in max_features:
    
    # GB
    max_score = 0
    for c in loss:
        for s in lr:
            for d in n_estimators:
                for f in criterion:
                    i += 1
                    # dtree = DecisionTreeClassifier(random_state=0, criterion=c, splitter=s, max_depth=d, max_features=f)
                    # DECISION TREES
                    # dtree.fit(X, Y, sample_weight=None, check_input=True)
                    # # out = dtree.predict(X, check_input=True)
                    # # print(out, Y)
                    # print(f"{i}) criterion: {c}, splitter: {s}, max_depth: {d}, max_features: {f} -> {dtree.score(testX, testY, sample_weight=None)}")

                    # BAGGING WITH DTREES
                    # bagg = BaggingClassifier(base_estimator=dtree, n_estimators=50, random_state=0).fit(X, Y)
                    # print(f"{i}) criterion: {c}, splitter: {s}, max_depth: {d}, max_features: {f} -> {bagg.score(testX, testY)}")

                    # RANDOM FORESTS
                    # randomforest = RandomForestClassifier(random_state=0, criterion=c, n_estimators=s, max_depth=d, max_features=f)
                    # randomforest.fit(X, Y, sample_weight=None)
                    # print(f"{i}) criterion: {c}, splitter: {s}, max_depth: {d}, max_features: {f} -> {randomforest.score(testX, testY, sample_weight=None)}")

                    #Gradient Boosting
                    gradientboost = GradientBoostingClassifier(loss=c, learning_rate=s, n_estimators=d, criterion=f)
                    gradientboost.fit(trainX, trainY, sample_weight=None)
                    score = gradientboost.score(testX, testY, sample_weight=None)
                    if score > max_score:
                        max_score = score
                        max_loss = c
                        max_lr = s
                        max_n_estimators = d
                        max_criterion = f
                    print(f"{i}) loss: {c}, lr: {s}, n_estimators: {d}, criterion: {f} -> {score}")
    
    print(f">>>) loss: {max_loss}, lr: {max_lr}, n_estimators: {max_n_estimators}, criterion: {max_criterion} -> {max_score}")
    gradientboost = GradientBoostingClassifier(loss=c, learning_rate=s, n_estimators=d, criterion=f)
    gradientboost.fit(trainX.append(tuneX), trainY.append(tuneY), sample_weight=None)
    score = gradientboost.score(testX, testY, sample_weight=None)
    print(f">>>) Train+Tune: loss: {max_loss}, lr: {max_lr}, n_estimators: {max_n_estimators}, criterion: {max_criterion} -> {max_score}")


# model(train_X, train_Y, test_X, test_Y)

In [60]:
clauses = [300, 500, 1000, 1500, 1800]
nsamples = [100, 1000, 5000]

In [ ]:
data_management('c1000_d5000')



DECISION TRESS


1) criterion: gini, splitter: best, max_depth: 2, max_features: auto -> 0.682
2) criterion: gini, splitter: best, max_depth: 2, max_features: sqrt -> 0.682
3) criterion: gini, splitter: best, max_depth: 2, max_features: log2 -> 0.7026
4) criterion: gini, splitter: best, max_depth: 5, max_features: auto -> 0.7546
5) criterion: gini, splitter: best, max_depth: 5, max_features: sqrt -> 0.7546
6) criterion: gini, splitter: best, max_depth: 5, max_features: log2 -> 0.7514
7) criterion: gini, splitter: best, max_depth: 7, max_features: auto -> 0.7997
8) criterion: gini, splitter: best, max_depth: 7, max_features: sqrt -> 0.7997
9) criterion: gini, splitter: best, max_depth: 7, max_features: log2 -> 0.769
10) criterion: gini, splitter: best, max_depth: 10, max_features: auto -> 0.78
11) criterion: gini, splitter: best, max_depth: 10, max_features: sqrt -> 0.78
12) criterion: gini, splitter: best, max_depth: 10, max_features: log2 -> 0.7853
13) criterion: gini, splitter: bes

In [ ]:
data_management('c1000_d1000')



DECISION TRESS


1) criterion: gini, splitter: best, max_depth: 2, max_features: auto -> 0.6845
2) criterion: gini, splitter: best, max_depth: 2, max_features: sqrt -> 0.6845
3) criterion: gini, splitter: best, max_depth: 2, max_features: log2 -> 0.711
4) criterion: gini, splitter: best, max_depth: 5, max_features: auto -> 0.745
5) criterion: gini, splitter: best, max_depth: 5, max_features: sqrt -> 0.745
6) criterion: gini, splitter: best, max_depth: 5, max_features: log2 -> 0.7485
7) criterion: gini, splitter: best, max_depth: 7, max_features: auto -> 0.7555
8) criterion: gini, splitter: best, max_depth: 7, max_features: sqrt -> 0.7555
9) criterion: gini, splitter: best, max_depth: 7, max_features: log2 -> 0.7415
10) criterion: gini, splitter: best, max_depth: 10, max_features: auto -> 0.7565
11) criterion: gini, splitter: best, max_depth: 10, max_features: sqrt -> 0.7565
12) criterion: gini, splitter: best, max_depth: 10, max_features: log2 -> 0.727
13) criterion: gini, splitter: 

In [ ]:
data_management('c1000_d100')



DECISION TRESS


1) criterion: gini, splitter: best, max_depth: 2, max_features: auto -> 0.705
2) criterion: gini, splitter: best, max_depth: 2, max_features: sqrt -> 0.705
3) criterion: gini, splitter: best, max_depth: 2, max_features: log2 -> 0.725
4) criterion: gini, splitter: best, max_depth: 5, max_features: auto -> 0.705
5) criterion: gini, splitter: best, max_depth: 5, max_features: sqrt -> 0.705
6) criterion: gini, splitter: best, max_depth: 5, max_features: log2 -> 0.715
7) criterion: gini, splitter: best, max_depth: 7, max_features: auto -> 0.705
8) criterion: gini, splitter: best, max_depth: 7, max_features: sqrt -> 0.705
9) criterion: gini, splitter: best, max_depth: 7, max_features: log2 -> 0.645
10) criterion: gini, splitter: best, max_depth: 10, max_features: auto -> 0.57
11) criterion: gini, splitter: best, max_depth: 10, max_features: sqrt -> 0.57
12) criterion: gini, splitter: best, max_depth: 10, max_features: log2 -> 0.595
13) criterion: gini, splitter: best, max_

In [ ]:
data_management('c1500_d5000')



DECISION TRESS


1) criterion: gini, splitter: best, max_depth: 2, max_features: auto -> 0.7876
2) criterion: gini, splitter: best, max_depth: 2, max_features: sqrt -> 0.7876
3) criterion: gini, splitter: best, max_depth: 2, max_features: log2 -> 0.7389
4) criterion: gini, splitter: best, max_depth: 5, max_features: auto -> 0.8506
5) criterion: gini, splitter: best, max_depth: 5, max_features: sqrt -> 0.8506
6) criterion: gini, splitter: best, max_depth: 5, max_features: log2 -> 0.8414
7) criterion: gini, splitter: best, max_depth: 7, max_features: auto -> 0.8874
8) criterion: gini, splitter: best, max_depth: 7, max_features: sqrt -> 0.8874
9) criterion: gini, splitter: best, max_depth: 7, max_features: log2 -> 0.859
10) criterion: gini, splitter: best, max_depth: 10, max_features: auto -> 0.9049
11) criterion: gini, splitter: best, max_depth: 10, max_features: sqrt -> 0.9049
12) criterion: gini, splitter: best, max_depth: 10, max_features: log2 -> 0.8827
13) criterion: gini, splitte

In [ ]:
data_management('c1500_d1000')



DECISION TRESS


1) criterion: gini, splitter: best, max_depth: 2, max_features: auto -> 0.766
2) criterion: gini, splitter: best, max_depth: 2, max_features: sqrt -> 0.766
3) criterion: gini, splitter: best, max_depth: 2, max_features: log2 -> 0.7365
4) criterion: gini, splitter: best, max_depth: 5, max_features: auto -> 0.83
5) criterion: gini, splitter: best, max_depth: 5, max_features: sqrt -> 0.83
6) criterion: gini, splitter: best, max_depth: 5, max_features: log2 -> 0.838
7) criterion: gini, splitter: best, max_depth: 7, max_features: auto -> 0.8805
8) criterion: gini, splitter: best, max_depth: 7, max_features: sqrt -> 0.8805
9) criterion: gini, splitter: best, max_depth: 7, max_features: log2 -> 0.847
10) criterion: gini, splitter: best, max_depth: 10, max_features: auto -> 0.8645
11) criterion: gini, splitter: best, max_depth: 10, max_features: sqrt -> 0.8645
12) criterion: gini, splitter: best, max_depth: 10, max_features: log2 -> 0.852
13) criterion: gini, splitter: best,

In [67]:
1

1

In [ ]:
data_management('c1500_d100')



DECISION TRESS


1) criterion: gini, splitter: best, max_depth: 2, max_features: auto -> 0.765
2) criterion: gini, splitter: best, max_depth: 2, max_features: sqrt -> 0.765
3) criterion: gini, splitter: best, max_depth: 2, max_features: log2 -> 0.77
4) criterion: gini, splitter: best, max_depth: 5, max_features: auto -> 0.83
5) criterion: gini, splitter: best, max_depth: 5, max_features: sqrt -> 0.83
6) criterion: gini, splitter: best, max_depth: 5, max_features: log2 -> 0.8
7) criterion: gini, splitter: best, max_depth: 7, max_features: auto -> 0.745
8) criterion: gini, splitter: best, max_depth: 7, max_features: sqrt -> 0.745
9) criterion: gini, splitter: best, max_depth: 7, max_features: log2 -> 0.795
10) criterion: gini, splitter: best, max_depth: 10, max_features: auto -> 0.745
11) criterion: gini, splitter: best, max_depth: 10, max_features: sqrt -> 0.745
12) criterion: gini, splitter: best, max_depth: 10, max_features: log2 -> 0.79
13) criterion: gini, splitter: best, max_dept

In [61]:
data_management('c1800_d5000')



DECISION TRESS


1) criterion: gini, splitter: best, max_depth: 2, max_features: auto -> 0.8015
2) criterion: gini, splitter: best, max_depth: 2, max_features: sqrt -> 0.8015
3) criterion: gini, splitter: best, max_depth: 2, max_features: log2 -> 0.8104
4) criterion: gini, splitter: best, max_depth: 5, max_features: auto -> 0.8919
5) criterion: gini, splitter: best, max_depth: 5, max_features: sqrt -> 0.8919
6) criterion: gini, splitter: best, max_depth: 5, max_features: log2 -> 0.8956
7) criterion: gini, splitter: best, max_depth: 7, max_features: auto -> 0.9416
8) criterion: gini, splitter: best, max_depth: 7, max_features: sqrt -> 0.9416
9) criterion: gini, splitter: best, max_depth: 7, max_features: log2 -> 0.9255
10) criterion: gini, splitter: best, max_depth: 10, max_features: auto -> 0.9645
11) criterion: gini, splitter: best, max_depth: 10, max_features: sqrt -> 0.9645
12) criterion: gini, splitter: best, max_depth: 10, max_features: log2 -> 0.9486
13) criterion: gini, splitt

In [65]:
data_management('c1800_d1000')



DECISION TRESS


1) criterion: gini, splitter: best, max_depth: 2, max_features: auto -> 0.7945
2) criterion: gini, splitter: best, max_depth: 2, max_features: sqrt -> 0.7945
3) criterion: gini, splitter: best, max_depth: 2, max_features: log2 -> 0.799
4) criterion: gini, splitter: best, max_depth: 5, max_features: auto -> 0.892
5) criterion: gini, splitter: best, max_depth: 5, max_features: sqrt -> 0.892
6) criterion: gini, splitter: best, max_depth: 5, max_features: log2 -> 0.902
7) criterion: gini, splitter: best, max_depth: 7, max_features: auto -> 0.9025
8) criterion: gini, splitter: best, max_depth: 7, max_features: sqrt -> 0.9025
9) criterion: gini, splitter: best, max_depth: 7, max_features: log2 -> 0.924
10) criterion: gini, splitter: best, max_depth: 10, max_features: auto -> 0.9335
11) criterion: gini, splitter: best, max_depth: 10, max_features: sqrt -> 0.9335
12) criterion: gini, splitter: best, max_depth: 10, max_features: log2 -> 0.9025
13) criterion: gini, splitter: b

In [ ]:
data_management('c1800_d100')



DECISION TRESS


1) criterion: gini, splitter: best, max_depth: 2, max_features: auto -> 0.77
2) criterion: gini, splitter: best, max_depth: 2, max_features: sqrt -> 0.77
3) criterion: gini, splitter: best, max_depth: 2, max_features: log2 -> 0.735
4) criterion: gini, splitter: best, max_depth: 5, max_features: auto -> 0.83
5) criterion: gini, splitter: best, max_depth: 5, max_features: sqrt -> 0.83
6) criterion: gini, splitter: best, max_depth: 5, max_features: log2 -> 0.8
7) criterion: gini, splitter: best, max_depth: 7, max_features: auto -> 0.83
8) criterion: gini, splitter: best, max_depth: 7, max_features: sqrt -> 0.83
9) criterion: gini, splitter: best, max_depth: 7, max_features: log2 -> 0.835
10) criterion: gini, splitter: best, max_depth: 10, max_features: auto -> 0.83
11) criterion: gini, splitter: best, max_depth: 10, max_features: sqrt -> 0.83
12) criterion: gini, splitter: best, max_depth: 10, max_features: log2 -> 0.81
13) criterion: gini, splitter: best, max_depth: 15

In [ ]:
data_management('c300_d5000')



DECISION TRESS


1) criterion: gini, splitter: best, max_depth: 2, max_features: auto -> 0.5884
2) criterion: gini, splitter: best, max_depth: 2, max_features: sqrt -> 0.5884
3) criterion: gini, splitter: best, max_depth: 2, max_features: log2 -> 0.589
4) criterion: gini, splitter: best, max_depth: 5, max_features: auto -> 0.6317
5) criterion: gini, splitter: best, max_depth: 5, max_features: sqrt -> 0.6317
6) criterion: gini, splitter: best, max_depth: 5, max_features: log2 -> 0.6179
7) criterion: gini, splitter: best, max_depth: 7, max_features: auto -> 0.6536
8) criterion: gini, splitter: best, max_depth: 7, max_features: sqrt -> 0.6536
9) criterion: gini, splitter: best, max_depth: 7, max_features: log2 -> 0.6188
10) criterion: gini, splitter: best, max_depth: 10, max_features: auto -> 0.6079
11) criterion: gini, splitter: best, max_depth: 10, max_features: sqrt -> 0.6079
12) criterion: gini, splitter: best, max_depth: 10, max_features: log2 -> 0.5916
13) criterion: gini, splitte

In [ ]:
data_management('c300_d1000')



DECISION TRESS


1) criterion: gini, splitter: best, max_depth: 2, max_features: auto -> 0.583
2) criterion: gini, splitter: best, max_depth: 2, max_features: sqrt -> 0.583
3) criterion: gini, splitter: best, max_depth: 2, max_features: log2 -> 0.5835
4) criterion: gini, splitter: best, max_depth: 5, max_features: auto -> 0.6045
5) criterion: gini, splitter: best, max_depth: 5, max_features: sqrt -> 0.6045
6) criterion: gini, splitter: best, max_depth: 5, max_features: log2 -> 0.5865
7) criterion: gini, splitter: best, max_depth: 7, max_features: auto -> 0.5895
8) criterion: gini, splitter: best, max_depth: 7, max_features: sqrt -> 0.5895
9) criterion: gini, splitter: best, max_depth: 7, max_features: log2 -> 0.5955
10) criterion: gini, splitter: best, max_depth: 10, max_features: auto -> 0.5715
11) criterion: gini, splitter: best, max_depth: 10, max_features: sqrt -> 0.5715
12) criterion: gini, splitter: best, max_depth: 10, max_features: log2 -> 0.5445
13) criterion: gini, splitter

In [ ]:
data_management('c300_d100')



DECISION TRESS


1) criterion: gini, splitter: best, max_depth: 2, max_features: auto -> 0.51
2) criterion: gini, splitter: best, max_depth: 2, max_features: sqrt -> 0.51
3) criterion: gini, splitter: best, max_depth: 2, max_features: log2 -> 0.52
4) criterion: gini, splitter: best, max_depth: 5, max_features: auto -> 0.475
5) criterion: gini, splitter: best, max_depth: 5, max_features: sqrt -> 0.475
6) criterion: gini, splitter: best, max_depth: 5, max_features: log2 -> 0.52
7) criterion: gini, splitter: best, max_depth: 7, max_features: auto -> 0.495
8) criterion: gini, splitter: best, max_depth: 7, max_features: sqrt -> 0.495
9) criterion: gini, splitter: best, max_depth: 7, max_features: log2 -> 0.585
10) criterion: gini, splitter: best, max_depth: 10, max_features: auto -> 0.495
11) criterion: gini, splitter: best, max_depth: 10, max_features: sqrt -> 0.495
12) criterion: gini, splitter: best, max_depth: 10, max_features: log2 -> 0.54
13) criterion: gini, splitter: best, max_dep

In [66]:
2

2

In [ ]:
data_management('c500_d5000')



DECISION TRESS


1) criterion: gini, splitter: best, max_depth: 2, max_features: auto -> 0.588
2) criterion: gini, splitter: best, max_depth: 2, max_features: sqrt -> 0.588
3) criterion: gini, splitter: best, max_depth: 2, max_features: log2 -> 0.6076
4) criterion: gini, splitter: best, max_depth: 5, max_features: auto -> 0.6576
5) criterion: gini, splitter: best, max_depth: 5, max_features: sqrt -> 0.6576
6) criterion: gini, splitter: best, max_depth: 5, max_features: log2 -> 0.6481
7) criterion: gini, splitter: best, max_depth: 7, max_features: auto -> 0.6838
8) criterion: gini, splitter: best, max_depth: 7, max_features: sqrt -> 0.6838
9) criterion: gini, splitter: best, max_depth: 7, max_features: log2 -> 0.6558
10) criterion: gini, splitter: best, max_depth: 10, max_features: auto -> 0.6625
11) criterion: gini, splitter: best, max_depth: 10, max_features: sqrt -> 0.6625
12) criterion: gini, splitter: best, max_depth: 10, max_features: log2 -> 0.6399
13) criterion: gini, splitter

In [ ]:
data_management('c500_d1000')



DECISION TRESS


1) criterion: gini, splitter: best, max_depth: 2, max_features: auto -> 0.5845
2) criterion: gini, splitter: best, max_depth: 2, max_features: sqrt -> 0.5845
3) criterion: gini, splitter: best, max_depth: 2, max_features: log2 -> 0.604
4) criterion: gini, splitter: best, max_depth: 5, max_features: auto -> 0.634
5) criterion: gini, splitter: best, max_depth: 5, max_features: sqrt -> 0.634
6) criterion: gini, splitter: best, max_depth: 5, max_features: log2 -> 0.622
7) criterion: gini, splitter: best, max_depth: 7, max_features: auto -> 0.647
8) criterion: gini, splitter: best, max_depth: 7, max_features: sqrt -> 0.647
9) criterion: gini, splitter: best, max_depth: 7, max_features: log2 -> 0.6185
10) criterion: gini, splitter: best, max_depth: 10, max_features: auto -> 0.617
11) criterion: gini, splitter: best, max_depth: 10, max_features: sqrt -> 0.617
12) criterion: gini, splitter: best, max_depth: 10, max_features: log2 -> 0.6065
13) criterion: gini, splitter: best

In [ ]:
data_management('c500_d100')



DECISION TRESS


1) criterion: gini, splitter: best, max_depth: 2, max_features: auto -> 0.52
2) criterion: gini, splitter: best, max_depth: 2, max_features: sqrt -> 0.52
3) criterion: gini, splitter: best, max_depth: 2, max_features: log2 -> 0.585
4) criterion: gini, splitter: best, max_depth: 5, max_features: auto -> 0.53
5) criterion: gini, splitter: best, max_depth: 5, max_features: sqrt -> 0.53
6) criterion: gini, splitter: best, max_depth: 5, max_features: log2 -> 0.625
7) criterion: gini, splitter: best, max_depth: 7, max_features: auto -> 0.56
8) criterion: gini, splitter: best, max_depth: 7, max_features: sqrt -> 0.56
9) criterion: gini, splitter: best, max_depth: 7, max_features: log2 -> 0.605
10) criterion: gini, splitter: best, max_depth: 10, max_features: auto -> 0.54
11) criterion: gini, splitter: best, max_depth: 10, max_features: sqrt -> 0.54
12) criterion: gini, splitter: best, max_depth: 10, max_features: log2 -> 0.505
13) criterion: gini, splitter: best, max_depth:

<h1>Extra MNIST</h1>

In [85]:
from sklearn.datasets import fetch_openml
X, y = fetch_openml("mnist_784", version=1, return_X_y=True)
X = X / 255

In [100]:
X_train, X_test = X[:10000], X[10000:12000]
y_train, y_test = y[:10000], y[10000:12000]

In [103]:
mnist_evaluation(X_train, y_train, X_test, y_test, dt[f][1:], bagg[f][1:], rf[f][1:], gb[f][1:])

DT>>>) Accuracy -> 0.5325
BAGG>>>) Accuracy -> 0.6875
RF>>>) Accuracy -> 0.6515
GB>>>) Accuracy -> 0.912


In [102]:
def mnist_evaluation(trainX, trainY, testX, testY, dtp, bagp, rfp, gbp):
    
    # DT
    dtree = DecisionTreeClassifier(random_state=0, criterion=dtp[0], splitter=dtp[1], max_depth=dtp[2], max_features=dtp[3])
    dtree.fit(trainX, trainY, sample_weight=None, check_input=True)
    predY = dtree.predict(testX)
    score = accuracy_score(testY, predY)
    # f1score = f1_score(testY, predY)
    print(f"DT>>>) Accuracy -> {score}")

    # BAG
    dtree = DecisionTreeClassifier(random_state=0, criterion=bagp[0], splitter=bagp[1], max_depth=bagp[2], max_features=bagp[3])
    dtree.fit(trainX, trainY, sample_weight=None, check_input=True)
    bagg = BaggingClassifier(base_estimator=dtree, n_estimators=50, random_state=0)
    bagg.fit(trainX, trainY)
    predY = bagg.predict(testX)
    score = accuracy_score(testY, predY)
    # f1score = f1_score(testY, predY)
    print(f"BAGG>>>) Accuracy -> {score}")

    #RF
    randomforest = RandomForestClassifier(random_state=0, criterion=rfp[0], n_estimators=50, max_depth=rfp[2], max_features=rfp[3])
    randomforest.fit(trainX, trainY, sample_weight=None)
    predY = randomforest.predict(testX)
    score = accuracy_score(testY, predY)
    # f1score = f1_score(testY, predY)
    print(f"RF>>>) Accuracy -> {score}")

    # GB
    gradientboost = GradientBoostingClassifier(loss="deviance", learning_rate=gbp[1], n_estimators=50, criterion="squared_error")
    gradientboost.fit(trainX, trainY, sample_weight=None)
    predY = gradientboost.predict(testX)
    score = accuracy_score(testY, predY)
    # f1score = f1_score(testY, predY)
    print(f"GB>>>) Accuracy -> {score}")

<h1> PART 2: Clustering</h1>

In [13]:
import os
import re
import numpy as np
import pandas as pd
import unicodedata
import math
import copy
from zipfile import ZipFile
import random
import imageio as iio

In [3]:
os.mkdir("/content/rhugaved_data1")
os.chdir("/content/rhugaved_data1")

After running above part, upload the zip folder in /content/rhugaved_data1 then run next cells

In [4]:
for filename in os.listdir("/content/rhugaved_data1"):
    if not os.path.isdir(filename):
        with ZipFile(filename, 'r') as zip:
            # extracting all the files
            zip.extractall()

In [5]:
!ls

hw3_part2_data.zip  KMeans.java  Koala.jpg  __MACOSX  Penguins.jpg


In [40]:
# ALL NUMPY NEW
def kmeans(rgb, k):
    cluster = np.array([-1 for i in range(rgb.shape[0])])
    k_list = np.array([[random.randint(0, 255) for j in range(3)]for i in range(k)])
    
    # Find the nearest neighbour first:
    k_list_count_prev = 0
    flag = True
    c = 0
    # Adding a hard bound to save time and not wait for convergence
    while flag and c < 25:
        c += 1
    # for x in range(20):
        # for i in range(len(rgb)):
        min = np.array([math.inf for i in range(rgb.shape[0])])
        for j in range(k):
            # pass
            # # Using NUMPY
            a = np.subtract(rgb, k_list[j])
            dist = np.sqrt(np.sum(np.multiply(a, a), axis=1))

            # Change the min array
            cluster[min > dist] = j
            # if sum(min <= dist):
            #     flag = False
            min[min > dist] = dist[min > dist]
            
            # print(min)
            # print(cluster)

        k_list_sum = [[0 for j in range(3)]for i in range(k)]
        k_list_count = [0 for i in range(k)]
        for i in range(len(rgb)):
            k_list_sum[cluster[i]] = np.add(k_list_sum[cluster[i]], rgb[i])
            k_list_count[cluster[i]] += 1
        for i in range(k):
            k_list[i] = np.floor_divide(k_list_sum[i], k_list_count[i])
        if k_list_count == k_list_count_prev:
            flag = False
        k_list_count_prev = k_list_count
    # print(rgb)

    print(k_list)
    print()
    return k_list, cluster



In [46]:
# Driver funtion
images = ['Koala.jpg', 'Penguins.jpg']
k_clusters = [2, 5, 10, 15, 20]
compression_ratio = dict()

for im in images:
    compression_ratio[im] = dict()
    img = iio.imread(im)
    original_size = os.path.getsize(im)
    # ALL NUMPY NEW
    rgb1 = np.empty((0, 3), int)
    for w in img:
        # print(w.shape)
        rgb1 = np.append(rgb1, np.array(w), axis=0)

    for k_c in k_clusters:
        compression_ratio[im][k_c] = []
        for n in range(3):
            rgb = copy.deepcopy(rgb1)
            print(f"For image = {im}, For K = {k} and n = {n}:")
            k_list, cluster = kmeans(rgb, k_c)

            for i in range(len(rgb)):
                rgb[i] = k_list[cluster[i]]

            rgb_np1 = np.zeros((768, 1024, 3))
            count = 0
            for w in range(img.shape[0]):
                for l in range(img.shape[1]):
                    for k in range(3):
                        rgb_np1[w][l][k] = rgb[count][k]
                    count += 1
            new_name = im.split('.')[0] + '_' + str(k_c) + '_' + str(n) + '.jpg'
            iio.imwrite(new_name, rgb_np1)
            new_size = os.path.getsize(new_name)
            compression_ratio[im][k_c].append(original_size/new_size)

For image = Koala.jpg, For K = 2 and n = 0:
[[ 55 251  71]
 [252 108  71]]


For image = Koala.jpg, For K = 2 and n = 1:
[[120  59 244]
 [188  87 207]]


For image = Koala.jpg, For K = 2 and n = 2:
[[113  63 183]
 [245  23  88]]


For image = Koala.jpg, For K = 2 and n = 0:
[[216 181  65]
 [135 253  59]
 [206  57 117]
 [ 67  42 150]
 [203 125 139]]


For image = Koala.jpg, For K = 2 and n = 1:
[[ 91 195 251]
 [ 60  62  27]
 [ 84  60 249]
 [179 121 102]
 [ 41 128  15]]


...

For image = Penguins.jpg, For K = 2 and n = 1:
[[ 46 141 209]
 [ 56 128  36]
 [ 65 186 211]
 [209 254  15]
 [ 93 237 124]
 [ 55 176   1]
 [100  44  23]
 [ 78  10 220]
 [233 104 164]
 [182 192 242]
 [208 173 154]
 [102 104  24]
 [232  40 150]
 [ 96 206 155]
 [160 138  26]]


For image = Penguins.jpg, For K = 2 and n = 2:
[[151 185  49]
 [155 212  49]
 [243 213  59]
 [ 94 238  69]
 [  4 232 186]
 [ 18 212  79]
 [ 75 198  30]
 [143 128  29]
 [202  51  33]
 [233  14  19]
 [102 100 133]
 [ 94 139  11]
 [ 14 102 189]
 [149 219 150]
 [ 83 155  17]]


For image = Penguins.jpg, For K = 2 and n = 0:
[[191 113  27]
 [180 140 192]
 [105 156 200]
 [235 242 121]
 [ 15 175 140]
 [ 37  51 142]
 [243 101 214]
 [242  56 185]
 [147 212 247]
 [122  15  60]
 [ 71  71 178]
 [ 74 183  93]
 [191 170 135]
 [104 153 225]
 [172 177 210]
 [ 74 144 209]
 [114 243 216]
 [ 44  64  85]
 [118  77 239]
 [199 246 127]]


For image = Penguins.jpg, For K = 2 and n = 1:
[[ 82 169 194]
 [214 150 118]
 [111 143 121]
 [ 90 238 120]
 [210 167 191]
 [217 244 220]
 [ 55 255  44]
 [149 148 241]
 [124 174  10]
 [248 175  65]
 [ 81  19  22]
 [131 146  48]
 [126 181 166]
 [142 184 159]
 [ 83  93 131]
 [ 91 108  32]
 [223 170 183]
 [ 55  85 113]
 [103  23 206]
 [ 61 111  68]]


For image = Penguins.jpg, For K = 2 and n = 2:
[[109   6 251]
 [195  12 247]
 [146  14  25]
 [132  39 184]
 [169 213 196]
 [ 88 103 138]
 [206 120  74]
 [158 166  28]
 [ 92  36  55]
 [219 140 128]
 [121 142 143]
 [ 16 125  37]
 [146 230 254]
 [160 101 153]
 [157 180 181]
 [252   2 196]
 [  2 178 218]
 [112 184  83]
 [202 189   1]
 [120 163 232]]


In [47]:
compression_ratio

{'Koala.jpg': {2: [4.273865757338573, 4.273865757338573, 4.261852260198457],
  5: [3.976143070287556, 3.9531143208638992, 3.98861389931806],
  10: [4.5480443836095175, 4.542963863693222, 4.553189380200711],
  15: [4.757567449001974, 4.776075308279506, 4.69393263560346],
  20: [4.806267350317922, 4.607623993154928, 4.677035040431267]},
 'Penguins.jpg': {2: [8.209080451278588, 8.209080451278588, 8.209080451278588],
  5: [7.154150379397563, 7.025878421100171, 7.107410453216374],
  10: [6.543301787592008, 6.474081534133471, 6.470903872551059],
  15: [6.478341259463466, 6.716823253082795, 6.724895171400164],
  20: [6.762371331200445, 6.716591254490191, 6.7120704830609395]}}

Calculating mean and variance for obtained compression rations

In [84]:
for im in compression_ratio:
    for k in compression_ratio[im]:
        summ = 0
        for i in compression_ratio[im][k]:
            summ += i
        mean = summ/3
        var = 0
        for i in compression_ratio[im][k]:
            var += (mean - i)**2
        var = var/3
        print(f"Image = {im}, k = {k}: MEAN = {mean} and VARIANCE = {var}")

Image = Koala.jpg, k = 2: MEAN = 4.269861258291868 and VARIANCE = 3.207202523012911e-05
Image = Koala.jpg, k = 5: MEAN = 3.972623763489839 and VARIANCE = 0.00021622943857208332
Image = Koala.jpg, k = 10: MEAN = 4.5480658758344825 and VARIANCE = 1.7427095598686913e-05
Image = Koala.jpg, k = 15: MEAN = 4.7425251309616465 and VARIANCE = 0.0012377054450738668
Image = Koala.jpg, k = 20: MEAN = 4.696975461301373 and VARIANCE = 0.006775340749735931
Image = Penguins.jpg, k = 2: MEAN = 8.209080451278588 and VARIANCE = 0.0
Image = Penguins.jpg, k = 5: MEAN = 7.095813084571369 and VARIANCE = 0.002809532027318694
Image = Penguins.jpg, k = 10: MEAN = 6.4960957314255126 and VARIANCE = 0.0011158887915858884
Image = Penguins.jpg, k = 15: MEAN = 6.640019894648809 and VARIANCE = 0.013080849848584574
Image = Penguins.jpg, k = 20: MEAN = 6.730344356250524 and VARIANCE = 0.000516269791273982
